# Text2SQL

In this notebook we explore how to generate a valid SQL query using LLMs. Some key modules include extracting relevant context and best practices in prompt engineering, to achieve desired outcome. 

First we show an example that does not work to illustrate how important database schema context is for the model to generate valid SQL code.

In the second example, the database schema is included in the prompt's context which results in a successful query execution.

In [ ]:
import inspect

## Bedrock Helpers

In [ ]:
%run ../utilities/bedrock_utils.py

## No context 

Here we demonstrate how important context is for the model to generate valid SQL queries. Without the database schema in the context, the model fails to generate valid SQL. 

Suggested questions to try:
1. How many games were played each year?
2. Give me a list of team names ordered by year it was founded.
3. Which teams where founded before 1960s and active in 2000s

In [ ]:
%run ../utilities/database_utils.py

In [ ]:
%run ../utilities/prompt_utils.py

In [ ]:
print(inspect.getsource(get_sql_query_prompt_no_context))

In [ ]:
print(inspect.getsource(get_sql_query_with_llm))

One can see in this example what happens when context is not given. The model generates what appears to be valid SQL, but would actually fail when executing against the database.
```python
# Generate SQL Query with no context
usr_query = "How many games were played each year?"
system_prompt, usr_msg = get_sql_query_prompt_no_context(usr_query)
sql_query = get_sql_query_with_llm(usr_msg, system_prompt , model_id=SONNET35_MODEL_ID)
print(sql_query)
```


```python
# Expected Output
"""SELECT YEAR(game_date) AS year, COUNT(*) AS games_played
FROM games
GROUP BY YEAR(game_date)
ORDER BY year;"""
```

```python
# Execute Query
if_executable, response = execute_sql(sql_query)
if not if_executable:
    print(response['Error'])
else:
    print(response)
```    

```python
# Expected Output
"""EXECUTING..ERROR: relation "games" does not exist"""
```

## With context

Here we demonstrate providing context in the form of table definitions. The table definitions include the table name, list of column names for each table and the type for each column. 

Suggested questions to try:
1. How many games were played each year?
2. Give me a list of team names ordered by year it was founded.
3. Which teams where founded before 1960s and active in 2000s

In [ ]:
schemas_dict = get_all_table_schema()
db_schema = "\n\n".join(schemas_dict)
print(db_schema)

In [ ]:
with open("db_schema.txt", "w") as f:
    f.write(db_schema)

In [ ]:
usr_query = "How many games were played each year?"
system_prompt, usr_msg = get_sql_query_prompt_generic_context(usr_query, db_schema)
sql_query = get_sql_query_with_llm(usr_msg, system_prompt, model_id=SONNET35_MODEL_ID)
print(sql_query)

In [ ]:
if_executable, response = execute_sql(sql_query)
if not if_executable:
    print(response['Error'])
else:
    print(response)

## Future extensions

* Relevant context
    * Challenge: when number of tables increases significantly, it might not fit in the context window or irrelevant tables may add noise.
    * Solution: Do a pre-processing step with less detailed view of the database (e.g., table summary rather than definition) to identify relevant tables
* Additional context
    * Challenge-1: Table definitions may not be sufficient to understand the content in the table (e.g. value format, unique values for categorical)
    * Solution-1: a) Provide few rows from each table as sample. b) Do a pre-processing step especially when WHERE clauses are involved
    * Challenge-2: Jargons in the table names or column names
    * Solution-2: Provide examples as additional context
* Enhancing query
    * Challenge: The user query may not be well informed. May require rewording to extract relevant columns from the tables.
    * Solution: Ask an LLM to rewrite the query and optimize